In [8]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [9]:
data = pd.read_csv('hcvdat0.csv')
data.columns

Index(['0', 'Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE',
       'CHOL', 'CREA', 'GGT', 'PROT'],
      dtype='object')

In [10]:
data.drop(columns='0', inplace=True, axis=1)
data['Category'] = data['Category'].str.replace(r'^[^=]*=', '', regex=True).str.strip().str.title()
data['Sex'] = data['Sex'].map({'m': 'Male', 'f': 'Female'})
data

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,Blood Donor,32,Male,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,Blood Donor,32,Male,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,Blood Donor,32,Male,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,Blood Donor,32,Male,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,Blood Donor,32,Male,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,Cirrhosis,62,Female,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
611,Cirrhosis,64,Female,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3
612,Cirrhosis,64,Female,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0
613,Cirrhosis,46,Female,33.0,NaN,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0


In [11]:
target = data[['Category']]
features = data.drop(columns=['Category'])
features = pd.concat(
    [features.drop(columns=['Sex']), pd.get_dummies(features['Sex'], prefix='is', drop_first=True).astype(int)],
    axis=1
) # SEX kolonunda one hot encoding uyguladım ancak true ve ya false döndürdüğü için sayısal değer alabilmek adına 1 veya 0 a çevirdim.


features.isna().sum() # Eksik verileri kontrol ettim, bazı kolonlarda eksik veriler var.
features

impute_subset = features[['Age', 'is_Male', 'ALP', 'CHOL', 'ALB', 'ALT', 'PROT']]
imputer = KNNImputer(n_neighbors=5)
imputed_array = imputer.fit_transform(impute_subset)
features['ALP'] = imputed_array[:, impute_subset.columns.get_loc('ALP')]
features['CHOL'] = imputed_array[:, impute_subset.columns.get_loc('CHOL')]
features['ALB'] = imputed_array[:, impute_subset.columns.get_loc('ALB')]
features['ALT'] = imputed_array[:, impute_subset.columns.get_loc('ALT')]
features['PROT'] = imputed_array[:, impute_subset.columns.get_loc('PROT')]
features.isna().sum() # hiç eksik veri kalmadı


Age        0
ALB        0
ALP        0
ALT        0
AST        0
BIL        0
CHE        0
CHOL       0
CREA       0
GGT        0
PROT       0
is_Male    0
dtype: int64

In [12]:
scaler = MinMaxScaler()
features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns, index=features.index)

In [13]:
import joblib
joblib.dump(scaler, 'atakan_scaler.pkl')


['atakan_scaler.pkl']

In [14]:
features

,Age,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,is_Male
0,0.224138,0.350669,0.101653,0.020962,0.036694,0.026461,0.367578,0.218447,0.091495,0.011757,0.535398,1.0
1,0.224138,0.350669,0.145571,0.052713,0.044990,0.012243,0.650434,0.408981,0.061619,0.017172,0.701327,1.0
2,0.224138,0.475483,0.156427,0.108816,0.134014,0.020932,0.494997,0.457524,0.072822,0.044400,0.763274,1.0
3,0.224138,0.420505,0.100419,0.091554,0.038290,0.071485,0.394263,0.401699,0.067221,0.045328,0.683628,1.0
4,0.224138,0.361070,0.154947,0.097719,0.045310,0.034755,0.515677,0.350728,0.063486,0.039295,0.528761,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
610,0.741379,0.254086,1.000000,0.015413,0.318124,0.194313,0.276851,0.591019,0.044534,1.000000,0.524336,0.0
611,0.775862,0.135215,0.225759,0.006165,0.107849,0.075829,0.008005,0.192961,0.051349,0.048577,0.586283,0.0
612,0.775862,0.209510,0.187515,0.008015,0.282068,0.186414,0.016011,0.266990,0.054803,0.092358,0.823009,0.0
613,0.465517,0.268945,0.163484,0.117448,0.164008,0.075829,0.142762,0.336165,0.041079,0.070390,0.579646,0.0


In [22]:
target_encoded

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
le = LabelEncoder()
target_encoded = le.fit_transform(target['Category'])

X_train, X_test, y_train, y_test = train_test_split(
    features, target_encoded, test_size=0.2, random_state=42, stratify=target_encoded
)

In [16]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'ANN (MLPClassifier)': MLPClassifier(hidden_layer_sizes=(120,), activation='relu', max_iter=1000, random_state=42)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    results[name] = {
        'Accuracy': report['accuracy'],
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score']
    }

/home/sercan/Genel/cse/backend/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [17]:
results_df = pd.DataFrame(results).T
results_df = results_df.round(3)
print(results_df)

                     Accuracy  Precision  Recall  F1-Score
Logistic Regression     0.886      0.807   0.886     0.843
Naive Bayes             0.870      0.918   0.870     0.891
KNN                     0.894      0.839   0.894     0.858
Decision Tree           0.894      0.894   0.894     0.892
Random Forest           0.927      0.901   0.927     0.913
SVM                     0.894      0.820   0.894     0.855
ANN (MLPClassifier)     0.959      0.964   0.959     0.950


In [18]:
mlp_model = MLPClassifier(hidden_layer_sizes=(120,), activation='relu', max_iter=1000, random_state=42)
mlp_model.fit(X_train, y_train)

joblib.dump(mlp_model, 'mlp_model.pkl')

/home/sercan/Genel/cse/backend/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


['mlp_model.pkl']

In [23]:
joblib.dump(le, 'label_encoder.pkl')

['label_encoder.pkl']

In [21]:
X_test

,Age,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,is_Male
71,0.327586,0.371471,0.127313,0.218249,0.106254,0.037915,0.632422,0.677184,0.084959,0.129641,0.626106,1.0
454,0.551724,0.347697,0.102640,0.035450,0.018826,0.011848,0.386925,0.485437,0.043880,0.012686,0.561947,0.0
344,0.275862,0.423477,0.126573,0.056412,0.036375,0.008689,0.574383,0.441748,0.052283,0.012222,0.619469,0.0
470,0.568966,0.323923,0.188256,0.102959,0.064454,0.066746,0.580387,0.673544,0.059752,0.030476,0.475664,0.0
558,0.637931,0.328380,0.253393,0.082922,1.000000,0.261453,0.288859,0.201456,0.083746,0.599783,0.719027,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
493,0.637931,0.294205,0.194917,0.067201,0.035099,0.010664,0.443629,0.487864,0.055084,0.006962,0.544248,0.0
443,0.517241,0.297177,0.065630,0.044390,0.028079,0.024882,0.258839,0.540049,0.070021,0.005260,0.391593,0.0
445,0.517241,0.421991,0.148532,0.084772,0.049138,0.021327,0.417612,0.543689,0.064420,0.022587,0.710177,0.0
80,0.344828,0.448737,0.129780,0.077374,0.034142,0.033570,0.476985,0.462379,0.066287,0.038521,0.584071,1.0
